In [27]:
import pymongo
from bs4 import BeautifulSoup as bs
from splinter import Browser
from webdriver_manager.chrome import ChromeDriverManager
import pandas as pd
import requests

In [28]:
#scrape the website https://mars.nasa.gov/news/ to scrape data on most recent headline and teaser associated with the headline.

url = 'https://mars.nasa.gov/news/'
response = requests.get(url)
bs_news= bs(response.text, 'html.parser')

#print(bs_news.prettify())

title_news = bs_news.find(class_='content_title').text.strip()
teaser_news = bs_news.find(class_='rollover_description').text.strip()

print(f'Title of the first news article is: "{title_news}"')
print(f'Article teaser: "{teaser_news}"')

Title of the first news article is: "NASA Ingenuity Mars Helicopter Prepares for First Flight"
Article teaser: "Now uncocooned from its protective carbon-fiber shield, the helicopter is being readied for its next steps."


In [29]:
#Next, we will use splinter to scrape the featured image on the website https://data-class-jpl-space.s3.amazonaws.com/JPL_Space/index.html

#setup splinter
executable_path={'executable_path': ChromeDriverManager().install()}
browser=Browser('chrome',**executable_path, headless=False)

#below, the url is separated into two parts in order to concatonate the "url" with the image path later.
url = 'https://data-class-jpl-space.s3.amazonaws.com/JPL_Space/'
index_url='index.html'
browser.visit(url+index_url)

[WDM] - ====== WebDriver manager ======
[WDM] - Current google-chrome version is 89.0.4389
[WDM] - Get LATEST driver version for 89.0.4389
[WDM] - Driver [C:\Users\kqo2\.wdm\drivers\chromedriver\win32\89.0.4389.23\chromedriver.exe] found in cache




In [30]:
#after inspecting the page, we see that the featured image is in the class "headerimage fade-in" as a src link within this img tag.

#use beautifulSoup to scrape the image link
html=browser.html
ftimg_soup = bs(html, 'html.parser')
ftimg_url = soup.find('img',class_="headerimage fade-in")
ftimg_url=ftimg_url.attrs['src'] #learned about the attrs function here: https://towardsdatascience.com/soup-of-the-day-97d71e6c07ec

#concatonate the image link with the earlier url link
featured_image_url=url+ftimg_url
browser.visit(featured_image_url)

#quit the browser session for splinter.
browser.quit()

In [31]:
print(featured_image_url)

https://data-class-jpl-space.s3.amazonaws.com/JPL_Space/image/featured/mars1.jpg
